# Übungsaufgaben 8

## Aufgabe 1 (Vergleich von POS-Taggern)

Laden Sie vier weitere 
[TCF-kodierte](https://weblicht.sfs.uni-tuebingen.de/weblichtwiki/index.php/The_TCF_Format)
Dateien herunter
(z.B. [Drude](https://www.deutschestextarchiv.de/book/download_fulltcf/16377),
[Sachs](https://www.deutschestextarchiv.de/book/download_fulltcf/16178),
[Brehm](https://www.deutschestextarchiv.de/book/download_fulltcf/25157)
und
[Bölsche](https://www.deutschestextarchiv.de/book/download_fulltcf/16552)).
Verwenden Sie diese Dateien und die von letzter Woche
([Altmann](https://www.deutschestextarchiv.de/book/download_fulltcf/16299))
um wie heute in der Vorlesung einen Bigramm-Tagger auf einer
Trainingsmenge zu trainieren und auf einer Testmenge auszuwerten.

Installieren Sie den TreeTagger und den Treetaggerwrapper und
schreiben Sie eine entsprechende Tagger-Klasse, die Token mit Hilfe
des TreeTaggers annotiert.  Werten Sie diesen Tagger auf der
Trainingsmenge aus und vergleichen Sie die Ergebnisse mit denen des
Bigramm-Taggers.


In [1]:
import sys
#!{sys.executable} -m pip install --upgrade pip nltk
#!{sys.executable} -m pip install --upgrade pip treetaggerwrapper
import nltk
import treetaggerwrapper as TTW 
import xml.etree.ElementTree as ET 

import logging 
logging.basicConfig(level=logging.INFO)

ns = {'tc': "http://www.dspin.de/data/textcorpus"}
urls = [
    'https://www.deutschestextarchiv.de/book/download_fulltcf/16377',
    'https://www.deutschestextarchiv.de/book/download_fulltcf/16178',
    'https://www.deutschestextarchiv.de/book/download_fulltcf/25157',
    'https://www.deutschestextarchiv.de/book/download_fulltcf/16552',
    'https://www.deutschestextarchiv.de/book/download_fulltcf/16299',
]   


In [2]:
def tags_from_tcf(root):
    token_ids = {t.attrib["ID"]: t.text for t in root.find('tc:TextCorpus', ns).find('tc:tokens', ns)}
    pos_tags = {t.attrib["tokenIDs"]: t.text for t in root.find('tc:TextCorpus', ns).find('tc:POStags', ns)}
    return [[(token_ids[id], pos_tags[id]) for id in sent.attrib['tokenIDs'].split(" ")] for sent in root.find('tc:TextCorpus', ns).find('tc:sentences', ns)]

In [3]:
import os

def read_xml_from_url(url):
    # Get the file name of the url.
    path = urllib.parse.urlparse(url)
    _, filename = os.path.split(path.path)
    filename += ".xml"
    # Check if sentence file is cached.
    if os.path.isfile(filename):
        logging.info(f"reading from cache: {filename}")
        with open(filename) as f:
            return f.read()
    # Download file.
    logging.debug(f"downloading from url: {url}")
    with urllib.request.urlopen(url) as f:
        contents = f.read()    
    logging.info(f"caching to file: {filename}")    
    with open(filename, 'wb') as f:
        f.write(contents)
    return contents


In [4]:
import urllib.request

def read_sentences_from_url(url):
    # Get the file name of the url.
    path = urllib.parse.urlparse(url)
    _, filename = os.path.split(path.path)
    filename += ".sents.txt"
    # Check if sentence file is cached.
    if os.path.isfile(filename):
        logging.info(f"reading from cache: {filename}")
        with open(filename) as f:
            # Read sentences from the cached file.
            return [[nltk.str2tuple(t) for t in sent[:-1].split(" ")] for sent in f.readlines()]

    # File is not cached; download it and cache it.
    sents = tags_from_tcf(ET.fromstring(read_xml_from_url(url)))
    logging.info(f"caching to file: {filename}")    
    with open(filename, 'w', encoding='utf-8') as f:
        for sent in sents:
            f.write(" ".join([t[0] + "/" + t[1] for t in sent]))
            f.write("\n")
    # Return sentences.
    return sents 

In [5]:
tagged_sents = [sent for url in urls for sent in read_sentences_from_url(url)]
print("number of sentences:", len(tagged_sents))

INFO:root:reading from cache: 16377.sents.txt
INFO:root:reading from cache: 16178.sents.txt
INFO:root:reading from cache: 25157.sents.txt
INFO:root:reading from cache: 16552.sents.txt
INFO:root:reading from cache: 16299.sents.txt


number of sentences: 42269


### Bigramm-Tagger mit Unigramm-Tagger als Backoff:

In [6]:
import random
random.seed(13)
random.shuffle(tagged_sents)
size = int(len(tagged_sents) * 0.9)
test_sents = tagged_sents[size:]
train_sents = tagged_sents[:size]
print("test set: ", len(test_sents))
print("train set:", len(train_sents))

test set:  4227
train set: 38042


#### Training und Evaluation:

In [7]:
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
t2.evaluate(test_sents)

0.9381128823782853

---
### Beispiel für Implementierung von eigenem POS-Tagger mit NLTK (mit SequentialBackoffTagger):

In [8]:
class LogTagger(nltk.tag.sequential.SequentialBackoffTagger):
    def __init__(self, backoff=None):
        super().__init__(backoff)

    def choose_tag(self, tokens, index, history):
        print("tokens: ", tokens)
        print("index:  ", index)
        print("history:", history)
        return None 

for sent in test_sents:
    if len(sent) == 5:
        short_sent = sent 

tx = LogTagger(backoff=t0)
tx.evaluate([short_sent])

tokens:  ['Hierzu', 'noch', 'einige', 'Erläuterungen', '.']
index:   0
history: []
tokens:  ['Hierzu', 'noch', 'einige', 'Erläuterungen', '.']
index:   1
history: ['NN']
tokens:  ['Hierzu', 'noch', 'einige', 'Erläuterungen', '.']
index:   2
history: ['NN', 'NN']
tokens:  ['Hierzu', 'noch', 'einige', 'Erläuterungen', '.']
index:   3
history: ['NN', 'NN', 'NN']
tokens:  ['Hierzu', 'noch', 'einige', 'Erläuterungen', '.']
index:   4
history: ['NN', 'NN', 'NN', 'NN']


0.2

### Vergleich mit TreeTagger:

#### Implementierung TreeTagger als NLTK-POS-Tagger (erbt von SequentialBackoffTagger) über TTW (Treetaggerwrapper):

In [ ]:
# Hilfsfunktion zur Umwandlung von TreeTagger-Tripel zu Tupel:
def tripple2tupple(str):
    tmp = str.split("\t")
    return (tmp[0], tmp[1])

In [10]:
logging.disable(logging.DEBUG)
logging.disable(logging.INFO)
logging.disable(logging.WARNING)

class TreeTagger(nltk.tag.sequential.SequentialBackoffTagger):
    def __init__(self, language, directory, backoff=None):
        super().__init__(backoff)
        self.tagger = TTW.TreeTagger(TAGLANG=language, TAGDIR=directory, TAGOPT='-token -sgml -quiet')
        self.tags = None

    def choose_tag(self, tokens, index, history):
        if index == 0: #neuer Satz
            self.tags = [tripple2tupple(tripple) for tripple in self.tagger.tag_text(" ".join(tokens))]
        if index < len(self.tags):
            return self.tags[index][1]
        return None

### ohne TAGOPT-Angaben: Fehler (TTW): Time out for TreeTagger (wegen Fehler in TreeTagger: hält Prozess an)
##Debugging: ohne lemma-Option in TAGOPT (TreeTagger-Lemmatisierung produziert Fehler bei bestimmten Tokens)

tt = TreeTagger('de', '../tree-tagger', t0)
tt.evaluate(test_sents)

0.7903317535545024

In [11]:
tt = TreeTagger('de', '../tree-tagger', t2)
tt.evaluate(test_sents)

0.7903317535545024

In [12]:
tt = TreeTagger('de', '../tree-tagger', t0)
t2 = nltk.BigramTagger(train_sents, backoff=tt)
t2.evaluate(test_sents)

0.8316415338216286

### Vergleich mit domänenfremden Testsets

- Treetagger: allgemeines deutsches Modell 
- hier trainierter Bigramm-Tagger: trainiert auf Korpus mit historischen zoologischen Büchern (deshalb besseres Ergebnis auf den entsprechenden Testdaten)

- jetzt: andere Testsets (domänenfremde Texte)

In [25]:
url='https://www.deutschestextarchiv.de/book/download_fulltcf/34066'
tagged_sents = read_sentences_from_url(url)

t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
t2.evaluate(tagged_sents)

0.8864010737599902

In [26]:
tt = TreeTagger('de', '../tree-tagger', t0)
tt.evaluate(tagged_sents)

0.8629125739735221

In [27]:
url = 'https://www.deutschestextarchiv.de/book/download_fulltcf/32274'
tagged_sents = read_sentences_from_url(url)
t2.evaluate(tagged_sents)

0.9168110918544194

In [28]:
tt.evaluate(tagged_sents)

0.9410745233968805